In [ ]:
import json
import os
import sys
from pathlib import Path

import jupyter_compare_view
import leafmap
import numpy as np
import rasterio
from samgeo import tms_to_geotiff


In [ ]:
%env SM_FRAMEWORK=tf.keras

# Add the parent directory to the path to make imports work
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

from predict import get_smooth_prediction_for_file
from utils import prepare_split_image

In [ ]:
with open("../data/geojson/wolfsburg_building_footprints_5km_reduced.geojson", "r") as f:
    data = json.load(f)

In [ ]:
dir_osm = Path("../data/osm")

# # wolfsburg
# string = "wolfsburg"
# center = [52.42119473870858, 10.789774079555059]
# zoom = 12
# zoom_save = 17

# Berlin
string = "berlin"
center = [52.487547, 13.437764]
zoom = 12
zoom_save = 17

In [ ]:
m = leafmap.Map(center=center, zoom=zoom)
m.add_basemap("SATELLITE")

# geo_json = leafmap.geojson_layer(data)
# m.add_layer(geo_json)
m

In [ ]:
# Get selection from map and download image
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()

    image = dir_osm / f"{string}_satellite-from-leafmap.tif"
    tms_to_geotiff(output=str(image), bbox=bbox, zoom=zoom_save, source="Satellite", overwrite=True)

else:
    print("No bounding box selected")

In [ ]:
# Get the prediction
model_path = Path(
    "../data/models/landcover_25_epochs_resnet34_backbone_batch16_iou_0.76.hdf5"
)

with rasterio.open(image) as dataset:
    img_array = dataset.read()

# Move channel information to third axis
img_array = np.moveaxis(img_array, source=0, destination=2)

prediction = get_smooth_prediction_for_file(
    img_array, model_path, 5, "resnet34", patch_size=256
)

In [ ]:
# Prepare images for visualization
img, segmented_img, overlay = prepare_split_image(img_array, prediction)

In [ ]:
# Show results
jupyter_compare_view.compare(
    img.convert("RGB"),
    overlay.convert("RGB"),
    start_mode="horizontal",
    start_slider_pos=0.5,
)